In [1]:
import csv
import os
from typing import Optional, Any

import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS, EVAL_DATALOADERS, STEP_OUTPUT, EPOCH_OUTPUT
from torch import nn
from torch.utils.data import DataLoader
from torchmetrics import Accuracy, Precision, Recall
from torchtime.datasets import PandasDataset
from torchtime.models import InceptionTime
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import numpy as np
from torchtime.transforms import ToTensor, Compose, LabelEncoder, NaN2Value

### Dataloading

In [2]:
df = pd.read_pickle("./eld_filtered_labled.pkl")


df['Consumption'] = df['Consumption'].map({0: 'low', 1: 'high'})
df = df.drop('day_of_week',axis=1)


In [3]:
df.Consumption.dtype

dtype('O')

In [4]:
dataset = PandasDataset(dataframe=df, labels="Consumption",
                                 target_transform=Compose([LabelEncoder(['low','high'])]))
print(dataset.dimensions)

None


In [5]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
print(test_dataset)

### Modified Inception module to fit the data

In [6]:
class InceptionTimeModule(pl.LightningModule):

    def __init__(self, df, num_workers: int = 8,
                 n_inputs: Optional[int] = None, n_classes: Optional[int] = None, use_residual=True,
                 use_bottleneck=True, depth=6, n_convolutions: int = 3, n_filters: int = 32, kernel_size=40,
                 learning_rate: int = 0.001, batch_size: int = 64, activation: str = 'linear', *args: Any, **kwargs: Any):
        super(InceptionTimeModule, self).__init__(*args, **kwargs)
        self.df = df
        self.num_workers = num_workers
        # self.generator = torch.Generator().manual_seed(42)

        # self.n_inputs = n_inputs
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.n_convolutions = n_convolutions
        self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.activation = activation

        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.ce_loss = nn.CrossEntropyLoss()

        self.train_acc = Accuracy(task="multiclass",num_classes=2)
        self.valid_acc = Accuracy(task="multiclass",num_classes=2)
        self.test_acc = Accuracy(task="multiclass",num_classes=2)
        self.test_precision = Precision(task="multiclass",num_classes=2)
        self.test_recall = Recall(task="multiclass",num_classes=2)

    @property
    def dims(self):
        return self.dataset.dim

    @property
    def n_classes(self):
        return len(self.dataset.classes)
#defining model parameters
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, betas=(0.9, 0.999), eps=1e-07,
                                     amsgrad=False)
        lr_scheduler = {"scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=50,
                                                                                min_lr=0.0001, mode='min'),
                        "monitor": "train_loss"}
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
#loading data using pandas module of library
    def setup(self, stage: Optional[str] = None) -> None:
        self.dataset = PandasDataset(dataframe=self.df,dimensions=list(self.df.columns)[:-1] ,labels="Consumption",
                                 transform=Compose([ToTensor(), NaN2Value()]),target_transform=Compose([LabelEncoder(['low','high'])]))
        train_size = int(0.8 * len(self.dataset))
        test_size = len(self.dataset) - train_size
        self.train_dataset, self.test_dataset = torch.utils.data.random_split(self.dataset, [train_size, test_size])

        self.model = InceptionTime(self.dims, self.n_classes, self.use_residual, self.use_bottleneck, self.depth,
                                   self.n_convolutions, self.n_filters, self.kernel_size, activation=self.activation)
    #data loader definations

    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(self.train_dataset, shuffle=True, batch_size=self.batch_size, num_workers=self.num_workers,
                          pin_memory=True, persistent_workers=True)

    def val_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(self.test_dataset, shuffle=False, batch_size=self.batch_size, num_workers=self.num_workers,
                          pin_memory=True, persistent_workers=True)

    def test_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(self.test_dataset, shuffle=False, batch_size=self.batch_size, num_workers=self.num_workers,
                          pin_memory=True)
#defining the steps of training,validation and testing for the module
    def training_step(self, batch, batch_idx) -> STEP_OUTPUT:
        x, targets = batch
        if torch.any(torch.isnan(x)):
            print("NaN")
        preds = self.model(x)
        loss = self.ce_loss(preds, targets)
        self.log("train_loss", loss)
        self.train_acc(preds, targets)
        self.log("train_acc", self.train_acc)
        return loss

    def validation_step(self, batch, batch_idx) -> Optional[STEP_OUTPUT]:
        x, targets = batch
        preds = self.model(x)
        loss = self.ce_loss(preds, targets)
        self.log("val_loss", loss)
        self.valid_acc(preds, targets)
        # self.precision(preds, targets)
        # self.recall(preds, targets)
        self.log("val_acc", self.valid_acc)
        # self.log("val_precision", self.precision)
        # self.log("val_recall", self.recall)
        return loss

    def test_step(self, batch, batchidx) -> Optional[STEP_OUTPUT]:
        x, targets = batch
        preds = self.model(x)
        loss = self.ce_loss(preds, targets)
        self.test_acc(preds, targets)
        self.test_precision(preds, targets)
        self.test_recall(preds, targets)
        self.log("test_loss", loss)
        self.log("test_acc", self.test_acc)
        return loss

    def test_epoch_end(self, outputs: EPOCH_OUTPUT) -> None:
        epoch_loss = torch.mean(torch.stack(outputs))
        print(epoch_loss.item(), self.test_acc.compute().item(), self.test_precision.compute().item(), self.test_recall.compute().item())

### Training

In [7]:
import warnings
warnings.filterwarnings('ignore')
model = InceptionTimeModule(df, num_workers=1)
checkpoint_callback = ModelCheckpoint(
            monitor="val_acc",
            dirpath=f"./models/",
            filename=f"inception_time",# + "{val_acc:.2f}",
            save_top_k=1,
            mode="max",
        )
early_stopping_callback = EarlyStopping(monitor="train_loss", mode="min", patience=50, min_delta=0.001,
                                        check_on_train_epoch_end=True)
logger = pl_loggers.TensorBoardLogger(save_dir=f"lightning_logs/")
lr_monitor = LearningRateMonitor(logging_interval='step')
trainer = Trainer(max_epochs=10, log_every_n_steps=1, accelerator="cpu",
                  check_val_every_n_epoch=5, enable_progress_bar=True, enable_model_summary=False, callbacks=[lr_monitor, checkpoint_callback,
                                                                                                             early_stopping_callback])
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [8]:
trainer.test()

Restoring states from the checkpoint path at C:\Users\puranik\Documents\Projects\torchtime-main\models\inception_time-v3.ckpt
Loaded model weights from checkpoint at C:\Users\puranik\Documents\Projects\torchtime-main\models\inception_time-v3.ckpt


Testing: 0it [00:00, ?it/s]

0.17559398710727692 0.9338728189468384 0.9338728189468384 0.9338728189468384
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9338728189468384
        test_loss           0.17485255002975464
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.17485255002975464, 'test_acc': 0.9338728189468384}]